# Tutorial: Meshes in FEniCS
### Written by Nicole Pitre for the Poulin Research Group
#### June 2016

This tutorial describes how to:
- Create a Gmsh script file (.geo file)
- Generate a finite element mesh from a .geo file
- Convert a mesh into XML format using dolfin-convert
- Mark subdomains of a mesh
- Include a mesh [and its subdomains] in a FEniCS demo

## Creating a Gmsh script file

The first step in generating a finite element mesh is to write a Gmsh script file that specifies the desired geometry. For more information on writing Gmsh script files including a list of all available geometry commands, see the Gmsh Reference Manual: http://gmsh.info/doc/texinfo/gmsh.html. 

We will start by looking at an example of a Gmsh script file called cape.geo. We will then open the file in Gmsh to see the physical representation of the code.


How will this code look as a geometric entity in Gmsh? We can open cape.geo in Gmsh to see how it looks. The result is shown below.
<img src="cape.jpg">

## Generating a finite element mesh from a Gmsh script file

We now want to generate a mesh file (.msh is the file extension) from our .geo file. 
In order to convert the cape.geo file into a mesh file, there are two options:
1. Use the Gmsh GUI. Open cape.geo in Gmsh and click Mesh --> 2D. Then click File --> Save Mesh. The mesh file, cape.msh, will appear in the same directory as the cape.geo file.
2. Use the following terminal command:

Note: If you are using Mac OS X, you may need to run the following Terminal command in order to run gmsh commands from the Terminal: 

The finite element mesh, cape.msh, is shown below.
<img src="cape_mesh.jpg">


## Converting a mesh into XML format using dolfin-convert

In order to import our cape mesh into FEniCS, we first need to convert it to XML format. The dolfin-convert Python script automatically converts a file from Gmsh format (.msh) to XML format, as shown in the cell below. The first argument is the input file and the second argument is the name of the output XML file.

In [1]:
%run dolfin-convert cape.msh cape.xml

Converting from Gmsh format (.msh, .gmsh) to DOLFIN XML format
Hello
Expecting 3306 vertices
Found all vertices
Expecting 6408 cells
Found all cells
Conversion done


The source code for dolfin-convert can be found at https://people.sc.fsu.edu/~jburkardt/py_src/dolfin-convert/dolfin-convert.py. To save the script to your machine, simply right click on the link and click "Save Link As..." and then select Python file. 

Further information about the dolfin-convert script can be found at https://people.sc.fsu.edu/~jburkardt/py_src/dolfin-convert/dolfin-convert.html


## Marking subdomains of a mesh

Before we import our XML-formatted mesh into FEniCS, we may want to write a Python script to mark subdomains of the mesh. Subdomains are necessary if we want to impose different boundary conditions on certain parts of the mesh boundary. The Python script shown below, cape_subdomains.py, marks five subdomains (top, bottom, right, left, interior) in the cape mesh. The output of the script is saved to an XML file called cape_subdomains.xml. The output is also saved to a VTK file that can be opened in visualization software (e.g. VisIt and ParaView). Note that only the XML file will be used in the final step of the tutorial.

Refer to documentation on the FEniCS website for help with the functions below. In particular, the following example is helpful: http://fenicsproject.org/documentation/dolfin/dev/python/demo/documented/subdomains/python/documentation.html

In [2]:
# %load cape_subdomains.py
from dolfin import *

set_log_level(1)

# Sub domain for top (mark whole boundary; bottom, right, left will overwrite)
class Top(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary

# Sub domain for bottom       
class Bottom(SubDomain):
    def inside(self, x, on_boundary):
        return x[1] < DOLFIN_EPS and on_boundary        

# Sub domain for right wall
class Right(SubDomain):
    def inside(self, x, on_boundary):
        return x[0] > 1.0 - DOLFIN_EPS and on_boundary

# Sub domain for left wall
class Left(SubDomain):
    def inside(self, x, on_boundary):
        return x[0] < DOLFIN_EPS and on_boundary

# Read mesh
mesh = Mesh("cape.xml")

# Create mesh functions over the cell facets
sub_domains = MeshFunction("size_t", mesh, mesh.topology().dim() - 1)

# Mark all facets as sub domain 4
sub_domains.set_all(4)

# Mark top as sub domain 0
top = Top()
top.mark(sub_domains, 0)

# Mark bottom as sub domain 1
bottom = Bottom()
bottom.mark(sub_domains, 1)

# Mark right wall as sub domain 2
right = Right()
right.mark(sub_domains, 2)

# Mark left wall as sub domain 3
left = Left()
left.mark(sub_domains, 3)

# Save sub domains to XML file
file = File("cape_subdomains.xml")
file << sub_domains

# Save sub domains to VTK files for visualization
file = File("cape_subdomains.pvd")
file << sub_domains


ImportError: No module named dolfin

## Including a mesh [and its subdomains] in a FEniCS demo

Now that we have the mesh and subdomain markers in XML format, we can add these files into a FEniCS demo. The following FEniCS code solves the Stokes equations using linear elements enriched with a bubble for the velocity and linear elements for the pressure (Mini elements). The output of the demo includes the norm of the velocity coefficient vector, the norm of the pressure coefficient vector, a VTK file for the velocity and a VTK file for the pressure. The VTK files can be viewed in VisIt or ParaView.

In [ ]:
# %load cape_stokes-mini.py
"""This demo solves the Stokes equations, using linear elements
enriched with a bubble for the velocity and linear elements for the
pressure (Mini elements)."""

# Copyright (C) 2007 Kristian B. Oelgaard
#
# This file is part of DOLFIN.
#
# DOLFIN is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# DOLFIN is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
# GNU Lesser General Public License for more details.
#
# You should have received a copy of the GNU Lesser General Public License
# along with DOLFIN. If not, see <http://www.gnu.org/licenses/>.
#
# Modified by Anders Logg, 2008-2009.
#
# First added:  2007-11-16
# Last changed: 2010-04-01
# Begin demo

from __future__ import print_function
from dolfin import *

# Load mesh and subdomains
mesh = Mesh("cape.xml")
sub_domains = MeshFunction("size_t", mesh, "cape_subdomains.xml")

# Define function spaces
P1 = VectorFunctionSpace(mesh, "Lagrange", 1)
B  = VectorFunctionSpace(mesh, "Bubble", 3)
Q  = FunctionSpace(mesh, "CG",  1)
V = P1 + B
Mini = V*Q

# Top boundary condition for velocity
# NOTE: Projection here is inefficient workaround of issue #489, FFC issue #69
top = project(Constant((0, 0)), V)
bc0 = DirichletBC(Mini.sub(0), top, sub_domains, 0)

# Right boundary condition for velocity
# NOTE: Projection here is inefficient workaround of issue #489, FFC issue #69
right = project(Expression(("-sin(x[1]*pi)", "0.0")), V)
bc1 = DirichletBC(Mini.sub(0), right, sub_domains, 2)

# Collect boundary conditions
bcs = [bc0, bc1]

# Define variational problem
(u, p) = TrialFunctions(Mini)
(v, q) = TestFunctions(Mini)
f = Constant((0, 0))
a = (inner(grad(u), grad(v)) - div(v)*p + q*div(u))*dx
L = inner(f, v)*dx

# Compute solution
w = Function(Mini)
solve(a == L, w, bcs)

# Split the mixed solution using deepcopy
# (needed for further computation on coefficient vector)
(u, p) = w.split(True)

print("Norm of velocity coefficient vector: %.15g" % u.vector().norm("l2"))
print("Norm of pressure coefficient vector: %.15g" % p.vector().norm("l2"))

# Split the mixed solution using a shallow copy
(u, p) = w.split()

# Save solution in VTK format
ufile_pvd = File("velocity.pvd")
ufile_pvd << u
pfile_pvd = File("pressure.pvd")
pfile_pvd << p

# Plot solution
plot(u)
plot(p)
interactive()
